# For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
# Author:Aljawharah Alareefi

# Import Libraries


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
import urllib.request # import the library we use to open URLs
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


# We scrape the Wikipedia page using Python, Urllib, Beautiful Soup and Pandas

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

In [4]:
#After parsing the html page we found the table with class wikitable sortable
torronto_table=soup.find('table', class_='wikitable sortable')

In [5]:
# Loop through the rows and store values into 3 list : A for postal code , B for Borough and C for Neighborhood
A=[]
B=[]
C=[]

for row in torronto_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [6]:
# Convert Lists to data frame
df_torronto = pd.DataFrame(A,columns=['PostalCode'])
df_torronto['Borough']=B
df_torronto['Neighborhood']=C
df_torronto.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [7]:
#remove \n 
df_torronto['PostalCode'] = df_torronto['PostalCode'].replace('\n','', regex=True)
df_torronto['Borough'] = df_torronto['Borough'].replace('\n','', regex=True)
df_torronto['Neighborhood'] = df_torronto['Neighborhood'].replace('\n','', regex=True)
df_torronto.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#drop rows with "Not Assigned" value and reindex and print the shap
df_torronto.drop(df_torronto[df_torronto['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)
df_torronto.reset_index(drop=True).head()
print(df_torronto.shape)
df_torronto.head()


(103, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# latitude and the longitude coordinates of each neighborhood. 

In [9]:
geo_ll = pd.read_csv('http://cocl.us/Geospatial_data')
geo_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Rename postal code column to match data frame column 
# Then merg to tables
geo_ll.rename(columns={'Postal Code':'PostalCode'},inplace=True)

geo_data = pd.merge(df_torronto, geo_ll, on='PostalCode')

In [12]:
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
